In [115]:
!pip3 install plotly
import plotly
import plotly.express as px
import os
import json
from statistics import mean
from collections import defaultdict
import matplotlib.pyplot as plt
import plotly.graph_objects as go

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
protasov = []
start_path2 = '/Users/Varvara/прога 2 курс/protasov/Probing_framework/results/experiment_2022_09_04-12:14:05_AM'
for root, dirs, files in os.walk(start_path2):
    for file in files:
        protasov.append(os.path.join(root, file))

В папке protasov лежит 14145 файлов json:

In [3]:
len(protasov[1:])

14145

Среди этих файлов встречается 104 языка и 80 категорий

In [124]:
languages = []
categories = []
for path in protasov[1:]:
    with open(path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    languages.append(data['params']['task_language'])
    categories.append(data['params']['task_category'])
        
print('Количество языков = ',len(set(languages)))
print('Количество категорий = ',len(set(categories)))

Количество языков =  104
Количество категорий =  80


Сколько файлов на каждый язык:

In [15]:
langs = set(languages)
catgrs  = set(categories)

In [131]:
lang_files = defaultdict(list)
for path in protasov[1:]:
    for lang in langs:
        if '_' + lang + '_' in path:
            lang_files[lang].append(path)
l = []
percent_f = []
for lang in lang_files:
    l.append(lang)
    percent_f.append(len(lang_files[lang]) / 14145)

In [143]:
fig = px.pie(names=l[:30], values=percent_f[:30], title='Кол-во файлов о языке в датасете')

# donut-like pie chart
fig.update_traces(hole=.3, hoverinfo="label+percent+name")

fig.update_layout(paper_bgcolor='white',
                  # Add annotations in the center of the donut pies.
                  annotations=[dict(text='Data', x=0.5, y=0.5, font_size=14, showarrow=False)])
fig.show()

Данные о том, сколько в каждом из 104 языка протестировано категорий:

In [152]:
lang_cat_file = defaultdict(dict)
for lang in lang_files:
    cat_files = defaultdict(list)
    for file in lang_files[lang]:
        for cat in catgrs:
            if cat in file:
                cat_files[cat].append(file)
    lang_cat_file[lang].update(cat_files)

#lang_cat_file
#словарь вида {язык1: {категория1: [файл1, файл2, ...], категори2: [файл1, файл2, ..]},
#              язык2: {категория1: [файл1, файл2, ...], ...},
#              ...}

la = []
has = []
other = []
lang_cat_file.keys()
for lang in lang_cat_file:
    #print(lang, len(lang_cat_file[lang].keys()) / 80)
    la.append(lang)
    has.append(len(lang_cat_file[lang].keys()))
    other.append(80 - len(lang_cat_file[lang].keys()))

In [162]:
fig = px.bar(x=la, y=[has, other], color_discrete_sequence=['purple', 'pink'], title="Cколько в каждом из 104 языка протестировано категорий")
fig.update_layout(xaxis_title="languages",
                  yaxis_title="amount of categories tested",)
fig.show()

Поскольку нет таких языков, в которых бы проверялись все категории, то для подсчета, какая категория лечше или хуже дается модели, необходимо определиться с кол-вом категорий, которые будем рассматривать в одних и тех же языках.

Данные о том, в каком кол-ве языков тестировалась категория:

In [58]:
cats_in_all_langs = defaultdict(list)
for lang in lang_cat_file:
    for cat in catgrs:
        if cat in lang_cat_file[lang].keys():
            cats_in_all_langs[cat].append(lang)
#cats_in_all_langs           

Чтобы определиться с категориями, можно взять категории, которые восстанавливались моделью в большем ко-ве языков, так как дальше можно будет легче найти пересечение среди языков этих категорий и получить материал, который можно визуализировать.

Допустим, установим порог: 70 языков – значит, данные, которые не заслуживают внимания – категории, которые были восстановлены меньше, чем в 70 языках.
Категории, которые много в каких языках протестировали:

In [111]:
freq_cats = []
freq_values = []
for cat in cats_in_all_langs:
    if len(cats_in_all_langs[cat]) > 70:
        print(cat, len(cats_in_all_langs[cat]))
        freq_cats.append(cat)
        freq_values.append(len(cats_in_all_langs[cat]))

Tense 83
PronType 79
VerbForm 75
Form 77
Number 92
Case 78
Person 91


In [123]:
fig = px.bar(x=freq_cats, y=freq_values, title='Категории, которые много в каких языках протестировали')
fig.update_layout(xaxis_title="categories",
                  yaxis_title="amount of languages")
fig.show()

Языки, в которых протестировали все 7 этих категорий:

In [80]:
inter_langs = set(cats_in_all_langs['Tense']).intersection(set(cats_in_all_langs['PronType']), set(cats_in_all_langs['VerbForm']), set(cats_in_all_langs['Form']), set(cats_in_all_langs['Number']), set(cats_in_all_langs['Case']), set(cats_in_all_langs['Person']))
len(inter_langs)

51

In [163]:
lang_cat_mean = defaultdict(dict)
for lang in inter_langs:
    for cat in lang_cat_file[lang]: #заходим в язык1 (из числа общих языков) и интерируемся по его категориям
        if cat in freq_cats: #если категория из числа тех 7
            av_layers_file = []
            cat_values = {}
            for path in lang_cat_file[lang][cat]: #открываем ее файлы
                with open(path, 'r', encoding='utf-8') as file:
                    data = json.load(file)
                #забираем среднее значение accuracy по всем 24 слоям
                average_layers = mean([val[0] for val in data['results']['test_score']['accuracy'].values()])
                av_layers_file.append(average_layers) #добавляем в список, где будут все средние по слоям во всех файлах категории 1 в языке 1
            #тут у нас список средних по всем файлам 1 категории 1 языка
            #берем и считаем среднее по файлам
            cat_values[cat] = mean(av_layers_file) # категория 1: ее среднее по файлам в языке 1
            #print(lang, cat_values)
        lang_cat_mean[lang].update(cat_values) 
    
#lang_cat_mean
#среднее accuracy по всем слоям
#во всех экспериментах про каждую из 7 категорий в каждом из 51 языка            

Среднее accuracy для 7 частотных категорий для каждого из 51 языка:

In [94]:
cat_mean_in_all_langs = defaultdict(list)
for lang in lang_cat_mean:
    for cat in lang_cat_mean[lang]:
        cat_mean_in_all_langs[cat].append(lang_cat_mean[lang][cat])

cats = []
means = []
for key, value in cat_mean_in_all_langs.items():
    cats.append(key)
    means.append(mean(value))
print(tuple(cats))
print(tuple(means))

('Number', 'Person', 'Case', 'VerbForm', 'Form', 'Tense', 'PronType')
(0.7118640672555961, 0.739239490769792, 0.6018679298571636, 0.695687835728738, 0.6831530798671936, 0.5546062693213409, 0.48434140671073284)


In [104]:
fig = px.scatter(x=cats, y=means, title="Какие категории даются машине лучше/хуже?", color_discrete_sequence=['purple'])
fig.update_layout(xaxis_title="categories",
                  yaxis_title="average_accuracy",)
fig.show()

На этом основании можно сделать вывод, что для языковой модели лучше получалось восстанавливать категорию Person и хуже всего категорию PronType.

Кстати, что вы вообще умеете из визуализации и js? – matplotlib, plotly, wordcloud чуточку LaTex. Из JavaScript, наверное, не могу сказать, но я готова научиться!

Как вам кажется, как нам совладать с такими объёмными экспериментами? – Выбрала бы иное хранение данных: DataFrame или Data Base. Очень долго собирать и группипровать информацию путем открытия и прочтения файлов. Мне кажется, что через pandas чуть менее кроваво собирать данные.